# Variable Expert Analysis - Unified Notebook

This notebook supports analysis of MoE models with variable-sized experts across different model sizes and expert configurations.

## Model Sizes

### GPT-2 120M (`DATASET = "gpt2"`)
- 12 layers, 768 dim, OpenWebText dataset
- **5:1 ratio** (`EXPERT_CONFIG = "5to1"`): 4x2560 + 4x512 experts
  - Available seeds: 42, 1223, 1337
- **23:1 ratio** (`EXPERT_CONFIG = "23to1"`): 4x2944 + 4x128 experts
  - Available seeds: 42, 1223, 1337

### GPT-2 250M (`DATASET = "gpt2_250m"`)
- 16 layers, 1024 dim, OpenWebText dataset (Chinchilla scaling)
- **5:1 ratio** (`EXPERT_CONFIG = "5to1"`): 4x3456 + 4x640 experts
  - Available seeds: 1337
- **31:1 ratio** (`EXPERT_CONFIG = "31to1"`): 4x3968 + 4x128 experts
  - Available seeds: 1337
- **Uniform baseline** (`EXPERT_CONFIG = "uniform"`): 8x2048 experts
  - Available seeds: 1337

### WikiText (`DATASET = "wikitext"`)
- 8 layers, 640 dim, WikiText dataset
- Single configuration: 4x2432 + 4x128 experts

## Usage

Change the configuration variables at the top of the config cell:

```python
DATASET = "gpt2"          # Choose model size: "gpt2", "gpt2_250m", "wikitext"
EXPERT_CONFIG = "5to1"    # Choose expert ratio: "5to1", "23to1", "31to1", "uniform"
SEED = 1337               # Choose seed: 42, 1223, 1337 (check availability above)
```

In [1]:
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from collections import defaultdict, Counter
from tqdm import tqdm
import tiktoken

from model import GPTConfig, MoeMLPWithTracking, GPTWithTracking


torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.set_float32_matmul_precision('high')  # Or 'medium' for even more speed

In [ ]:
# ========== CONFIGURATION ==========
# Change this to switch between model sizes and configurations
DATASET = "gpt2"  # Options: "gpt2" (120M), "gpt2_250m" (250M), "wikitext"
EXPERT_CONFIG = "23to1"  # Options: "5to1", "23to1", "31to1", "uniform" (depends on DATASET - see availability below)
SEED = 42 # Options: 42, 1223, 1337 (depending on which are available)

# Dataset-specific configurations
CONFIGS = {
    "gpt2": {
        "n_layer": 12,
        "n_head": 12,
        "n_embd": 768,
        "vocab_size": 50304,
        "val_data_path": "data/openwebtext/val.bin",
        "expert_configs": {
            "5to1": {
                "expert_sizes": [(4, 2560), (4, 512)],  # 5:1 ratio
                "base_dir": "gpt2_experiments/multiseed_5to1",
                "run_name_pattern": "ratio5_lbl0.01_compute0.004_seed{seed}",
                "model_name": "GPT-2 120M 5:1 (4x2560 + 4x512)",
                "available_seeds": [42, 1223, 1337]
            },
            "23to1": {
                "expert_sizes": [(4, 2944), (4, 128)],  # 23:1 ratio
                "base_dir": "gpt2_experiments/multiseed_23to1",
                "run_name_pattern": "ratio23_lbl0.01_compute0.004_seed{seed}",
                "model_name": "GPT-2 120M 23:1 (4x2944 + 4x128)",
                "available_seeds": [42, 1223, 1337]
            }
        }
    },
    "gpt2_250m": {
        "n_layer": 16,
        "n_head": 16,
        "n_embd": 1024,
        "vocab_size": 50304,
        "val_data_path": "data/openwebtext/val.bin",
        "expert_configs": {
            "5to1": {
                "expert_sizes": [(4, 3456), (4, 640)],  # 5:1 ratio
                "base_dir": "gpt2_250m_experiments/expert_sizes_final_weights",
                "run_name_pattern": "sizes_5to1_lbl0.01_compute0.004_seed{seed}",
                "model_name": "GPT-2 250M 5:1 (4x3456 + 4x640)",
                "available_seeds": [1337]
            },
            "31to1": {
                "expert_sizes": [(4, 3968), (4, 128)],  # 31:1 ratio
                "base_dir": "gpt2_250m_experiments/expert_sizes_final_weights",
                "run_name_pattern": "sizes_31to1_lbl0.01_compute0.004_seed{seed}",
                "model_name": "GPT-2 250M 31:1 (4x3968 + 4x128)",
                "available_seeds": [1337]
            },
            "uniform": {
                "expert_sizes": [(8, 2048)],  # 1:1 baseline
                "base_dir": "gpt2_250m_experiments/expert_sizes_final_weights",
                "run_name_pattern": "sizes_uniform_lbl0.01_compute0.004_seed{seed}",
                "model_name": "GPT-2 250M Uniform (8x2048)",
                "available_seeds": [1337]
            }
        }
    },
    "wikitext": {
        "n_layer": 8,
        "n_head": 8,
        "n_embd": 640,
        "vocab_size": 8192,
        "expert_sizes": [(4, 2432), (4, 128)],
        "checkpoint_dir": f"out-wikitext/moe-8x2-variable-4x2432-4x128-seed{SEED}",
        "val_data_path": "data/wikitext/val.bin",
        "model_name": f"WikiText (4x2432 + 4x128) seed{SEED}"
    }
}

# Build configuration based on DATASET
if DATASET in ["gpt2", "gpt2_250m"]:
    base_cfg = CONFIGS[DATASET]
    expert_cfg = base_cfg["expert_configs"][EXPERT_CONFIG]
    
    # Check if seed is available
    if SEED not in expert_cfg["available_seeds"]:
        print(f"WARNING: Seed {SEED} not available for {DATASET} {EXPERT_CONFIG}")
        print(f"Available seeds: {expert_cfg['available_seeds']}")
    
    # Build run name with seed
    if "{seed}" in expert_cfg["run_name_pattern"]:
        run_name = expert_cfg["run_name_pattern"].replace("{seed}", str(SEED))
    else:
        run_name = expert_cfg["run_name_pattern"]
    
    cfg = {
        "n_layer": base_cfg["n_layer"],
        "n_head": base_cfg["n_head"],
        "n_embd": base_cfg["n_embd"],
        "vocab_size": base_cfg["vocab_size"],
        "expert_sizes": expert_cfg["expert_sizes"],
        "checkpoint_dir": f"{expert_cfg['base_dir']}/{run_name}",
        "val_data_path": base_cfg["val_data_path"],
        "model_name": f"{expert_cfg['model_name']} seed{SEED}"
    }
else:
    # Flat config (e.g., wikitext)
    cfg = CONFIGS[DATASET]

# Create model config
config = GPTConfig(
    n_layer = cfg['n_layer'],
    n_head = cfg['n_head'],
    n_embd = cfg['n_embd'],
    bias = False,
    vocab_size= cfg['vocab_size'],
    
    # MoE configuration with VARIABLE-SIZE EXPERTS
    use_moe = True,
    num_experts = 8,
    num_experts_per_tok = 2,
    norm_topk_prob = True,
    block_size = 128,
    block_k = 64,
    expert_sizes = cfg["expert_sizes"]
)

print(f"\n{'='*60}")
print(f"Configuration: {cfg['model_name']}")
print(f"Expert sizes: {config.expert_sizes}")
print(f"Checkpoint: {cfg['checkpoint_dir']}/ckpt.pt")
print(f"Val data: {cfg['val_data_path']}")
print(f"{'='*60}\n")


Configuration: GPT-2 120M 5:1 (4x2560 + 4x512) seed42
Expert sizes: [(4, 2560), (4, 512)]
Checkpoint: gpt2_experiments/multiseed_5to1/ratio5_lbl0.01_compute0.004_seed42/ckpt.pt
Val data: data/openwebtext/val.bin



## Load Model and Checkpoint

In [ ]:
checkpoint_path = f"{cfg['checkpoint_dir']}/ckpt.pt"

model = GPTWithTracking(config).to(torch.bfloat16)

for block in model.transformer.h:
    if hasattr(block.mlp, 'expert_sizes'):
        old_mlp = block.mlp
        block.mlp = MoeMLPWithTracking(config).to(torch.bfloat16)
        block.mlp.load_state_dict(old_mlp.state_dict())

checkpoint = torch.load(checkpoint_path, map_location='cpu')

state_dict = checkpoint['model']
if any(k.startswith('_orig_mod.') for k in state_dict.keys()):
    state_dict = {k.replace('_orig_mod.', ''): v for k, v in state_dict.items()}

model.load_state_dict(state_dict)

print(f"✓ Loaded checkpoint from {checkpoint_path}")

## Collect ALL Routing Statistics in a SINGLE Pass

This cell collects all statistics (individual expert assignments and combinations) in one efficient pass through the validation set.

In [ ]:
tokenizer = tiktoken.get_encoding('gpt2')

val_data_path = cfg['val_data_path']
val_data = np.memmap(val_data_path, dtype=np.uint16, mode='r')

device = 'cuda'
model = model.to(device)
model.eval()

num_layers = config.n_layer
expert_sizes = model.transformer.h[0].mlp.expert_sizes

# Initialize ALL tracking structures
token_stats_per_layer = {}
token_combinations_per_layer = {}

for layer_idx in range(num_layers):
    layer_name = f'layer_{layer_idx}'
    token_stats_per_layer[layer_name] = defaultdict(lambda: {
        'expert_counts': np.zeros(config.num_experts, dtype=np.int64),
        'total_occurrences': 0,
        'total_entropy': 0.0,
        'total_layer_entropy': 0.0,
        'expert_size_sum': 0.0,
    })
    token_combinations_per_layer[layer_name] = defaultdict(Counter)

batch_size = 1
seq_len = 1024
total_tokens = len(val_data)
num_batches = total_tokens // seq_len

print(f"Running single pass through {num_batches} batches to collect all statistics...")

for batch_idx in tqdm(range(num_batches)):
    start_idx = batch_idx * seq_len
    end_idx = start_idx + seq_len
    batch_tokens = torch.from_numpy(val_data[start_idx:end_idx].astype(np.int64)).unsqueeze(0).to(device)

    with torch.inference_mode():
        logits, loss, aux_loss = model(batch_tokens, targets=batch_tokens)

    output_probs = F.softmax(logits[0], dim=-1)
    epsilon = 1e-10
    output_entropy = -(output_probs * torch.log(output_probs + epsilon)).sum(dim=-1).float().cpu().numpy()

    for layer_idx in range(num_layers):
        layer_name = f'layer_{layer_idx}'
        layer_assignments = aux_loss['expert_assignments'][layer_name][0].cpu().numpy()
        layer_entropies = aux_loss['layer_entropies'][layer_name][0].cpu().numpy()
        token_stats = token_stats_per_layer[layer_name]

        for pos in range(seq_len):
            token_id = int(batch_tokens[0, pos].item())
            expert_ids = layer_assignments[pos]

            # Update individual expert statistics
            token_stats[token_id]['total_occurrences'] += 1
            token_stats[token_id]['total_entropy'] += output_entropy[pos]
            token_stats[token_id]['total_layer_entropy'] += layer_entropies[pos]

            for expert_id in expert_ids:
                token_stats[token_id]['expert_counts'][expert_id] += 1
                token_stats[token_id]['expert_size_sum'] += expert_sizes[expert_id]

            # Track expert combinations
            expert_combination = tuple(sorted(expert_ids))
            token_combinations_per_layer[layer_name][token_id][expert_combination] += 1

print(f"\n✓ Collected all statistics in a single pass!")

## Analysis: Print the Most Common Expert Combination for Each Token

This shows which expert pair each token uses most frequently across all layers.

In [ ]:
def format_expert_combo(combo, expert_sizes):
    """Format expert combination with sizes"""
    return ",".join([f"{e}({expert_sizes[e]})" for e in combo])

def print_token_routing_table(token_combinations_per_layer, token_stats_per_layer, expert_sizes, tokenizer, max_tokens=100):
    """Print comprehensive routing table"""
    
    # Calculate average expert size and FLOPs for each token
    token_avg_sizes = {}
    for layer_name in token_stats_per_layer:
        for token_id, stats in token_stats_per_layer[layer_name].items():
            if stats['total_occurrences'] > 0:
                avg_size = stats['expert_size_sum'] / (stats['total_occurrences'] * 2)  # 2 experts per token
                if token_id not in token_avg_sizes:
                    token_avg_sizes[token_id] = []
                token_avg_sizes[token_id].append(avg_size)
    
    overall_avg_sizes = {tid: np.mean(sizes) for tid, sizes in token_avg_sizes.items()}
    
    # Header
    print(f"\n{'='*200}")
    header = f"{'Token ID':<10}{'Token':<20}{'Avg Size':<13}{'FLOPs':<16}"
    for layer_idx in range(num_layers):
        header += f"{'Layer ' + str(layer_idx):<21}"
    print(header)
    print(f"{'='*200}")
    
    # Print first N tokens
    for token_id in sorted(overall_avg_sizes.keys())[:max_tokens]:
        try:
            token_str = tokenizer.decode([token_id]).replace('\n', '\\n')
        except:
            token_str = f"<{token_id}>"
        
        avg_size = overall_avg_sizes[token_id]
        # FLOPs calculation: 2 * seq_len * hidden * expert_size (for matrix mult)
        # Simplified: avg_size * hidden * 2 operations
        flops = avg_size * config.n_embd * 4 * config.n_layer  # 4x forward/backward
        
        row = f"{token_id:<10}{token_str:<20}{avg_size:<13.1f}{flops:<16,.0f}"
        
        for layer_idx in range(num_layers):
            layer_name = f'layer_{layer_idx}'
            if token_id in token_combinations_per_layer[layer_name]:
                combos = token_combinations_per_layer[layer_name][token_id]
                most_common = combos.most_common(1)[0][0]
                combo_str = f"({format_expert_combo(most_common, expert_sizes)})"
                row += f"{combo_str:<21}"
            else:
                row += f"{'N/A':<21}"
        
        print(row)

print_token_routing_table(token_combinations_per_layer, token_stats_per_layer, expert_sizes, tokenizer, max_tokens=350)

In [ ]:
import pandas as pd

print("Building dataframe from collected statistics...")

# Get unique tokens
all_token_ids = set()
for layer_combos in token_combinations_per_layer.values():
    all_token_ids.update(layer_combos.keys())

# Build data for dataframe
data = []
for token_id in all_token_ids:
    # Decode token
    try:
        token_text = tokenizer.decode([token_id])
        token_text = token_text.replace('\n', '\\n').replace('\t', '\\t').replace('\r', '\\r')
        if '�' in token_text or not token_text.isprintable():
            token_text = f"<{token_id}>"
        if len(token_text) > 18:
            token_text = token_text[:17] + '…'
    except:
        token_text = f"<{token_id}>"
    
    # Calculate average expert SIZE and ENTROPY across all layers
    total_size = 0
    total_entropy = 0.0
    layer_count = 0
    layer_data = {}
    layer_entropy_data = {}
    token_count = 0  # Track total occurrences of this token
    
    for layer_idx in range(num_layers):
        layer_name = f'layer_{layer_idx}'
        combos = token_combinations_per_layer[layer_name][token_id]
        stats = token_stats_per_layer[layer_name][token_id]
        
        # Get token count from the first layer (all layers have same count)
        if layer_idx == 0 and stats['total_occurrences'] > 0:
            token_count = stats['total_occurrences']
        
        if combos:
            most_common = combos.most_common(1)[0][0]
            layer_size = sum(expert_sizes[e] for e in most_common)
            total_size += layer_size
            
            # Calculate average entropy for this token in this layer
            if stats['total_occurrences'] > 0:
                layer_entropy = stats['total_entropy'] / stats['total_occurrences']
                total_entropy += layer_entropy
                
                # Store layer-wise intermediate entropy
                layer_wise_entropy = stats['total_layer_entropy'] / stats['total_occurrences']
                layer_entropy_data[f'layer_{layer_idx}_entropy'] = layer_wise_entropy
            
            layer_count += 1
            # Format with expert sizes: (5(128),7(128))
            formatted = "(" + ",".join([f"{e}({expert_sizes[e]})" for e in most_common]) + ")"
            layer_data[f'layer_{layer_idx}'] = formatted
        else:
            layer_data[f'layer_{layer_idx}'] = 'N/A'
            layer_entropy_data[f'layer_{layer_idx}_entropy'] = np.nan
    
    avg_size = total_size / layer_count if layer_count > 0 else 0
    avg_entropy = total_entropy / layer_count if layer_count > 0 else 0
    flops = 4 * config.n_embd * total_size
    
    row = {
        'token_id': token_id,
        'token': token_text,
        'count': token_count,
        'avg_size': avg_size,
        'avg_entropy': avg_entropy,
        'flops': flops,
        **layer_data,
        **layer_entropy_data
    }
    data.append(row)

# Create DataFrame and sort by FLOPs
df = pd.DataFrame(data)
df = df.sort_values('flops', ascending=True).reset_index(drop=True)

# Calculate weighted FLOPs based on token frequency
total_dataset_flops = (df['flops'] * df['count']).sum()
total_tokens = df['count'].sum()
weighted_avg_flops = total_dataset_flops / total_tokens

print(f"\nDataFrame created with {len(df)} tokens, sorted by FLOPs (low to high)")
print(f"\nFirst 20 rows (lowest FLOPs):")
# Select a subset of columns for display
display_cols = ['token_id', 'token', 'count', 'avg_size', 'avg_entropy', 'flops', 
                'layer_0_entropy', 'layer_5_entropy', 'layer_11_entropy']
print(df[display_cols].head(20).to_string())

print(f"\n\nLast 20 rows (highest FLOPs):")
print(df[display_cols].tail(20).to_string())

# Summary statistics
print(f"\n{'='*80}")
print(f"Summary Statistics:")
print(f"{'='*80}")
print(f"Total unique tokens: {len(df)}")
print(f"Total token occurrences: {total_tokens:,}")

print(f"\nToken Count distribution:")
print(f"  Min:    {df['count'].min():,}")
print(f"  25%:    {df['count'].quantile(0.25):,.0f}")
print(f"  Median: {df['count'].median():,.0f}")
print(f"  75%:    {df['count'].quantile(0.75):,.0f}")
print(f"  Max:    {df['count'].max():,}")
print(f"  Mean:   {df['count'].mean():,.0f}")

print(f"\nFLOPs distribution:")
print(f"  Min:    {df['flops'].min():,.0f}")
print(f"  25%:    {df['flops'].quantile(0.25):,.0f}")
print(f"  Median: {df['flops'].median():,.0f}")
print(f"  75%:    {df['flops'].quantile(0.75):,.0f}")
print(f"  Max:    {df['flops'].max():,.0f}")
print(f"  Mean:   {df['flops'].mean():,.0f}")

print(f"\nAverage Size distribution:")
print(f"  Tokens with avg_size >= {4*config.n_embd}: {(df['avg_size'] >= 4*config.n_embd).sum()} ({100*(df['avg_size'] >= config.n_embd).sum()/len(df):.2f}%)")
print(f"  Tokens with avg_size < {4*config.n_embd}:  {(df['avg_size'] < 4*config.n_embd).sum()} ({100*(df['avg_size'] < 4*config.n_embd).sum()/len(df):.2f}%)")

print(f"\nAverage Output Entropy distribution:")
print(f"  Min:    {df['avg_entropy'].min():.4f}")
print(f"  25%:    {df['avg_entropy'].quantile(0.25):.4f}")
print(f"  Median: {df['avg_entropy'].median():.4f}")
print(f"  75%:    {df['avg_entropy'].quantile(0.75):.4f}")
print(f"  Max:    {df['avg_entropy'].max():.4f}")
print(f"  Mean:   {df['avg_entropy'].mean():.4f}")

print(f"\nLayer-wise Entropy Statistics:")
for layer_idx in [0, num_layers//2, num_layers-1]:
    layer_col = f'layer_{layer_idx}_entropy'
    print(f"  Layer {layer_idx}:")
    print(f"    Mean:   {df[layer_col].mean():.4f}")
    print(f"    Median: {df[layer_col].median():.4f}")
    print(f"    Std:    {df[layer_col].std():.4f}")

baseline_flops = config.n_layer * 4 * config.n_embd * (config.n_embd*4)  # num_layers * 4 * hidden_size * total expert_size
print(f"\n{'='*80}")
print(f"FLOPs Analysis:")
print(f"{'='*80}")
print(f"Average FLOPs per unique token: {df['flops'].mean():.0f} ({100*df['flops'].mean()/baseline_flops:.2f}% of baseline)")
print(f"Weighted average FLOPs per token occurrence: {weighted_avg_flops:.0f} ({100*weighted_avg_flops/baseline_flops:.2f}% of baseline)")
print(f"Total dataset FLOPs: {total_dataset_flops:,.0f}")
print(f"Baseline total FLOPs: {baseline_flops * total_tokens:,.0f}")
print(f"FLOPs savings: {100 * (1 - total_dataset_flops / (baseline_flops * total_tokens)):.2f}%")

# Store the dataframe for further analysis
expert_combinations_df = df
sweep_value = '-'.join(checkpoint_path.split('-')).split('/')[-2]

df.to_csv(f'analysis_csvs/{sweep_value}_expert_combinations.csv', index=False)
print(f"\n✓ Saved to analysis_csvs/{sweep_value}_expert_combinations.csv")
print(f"  Columns: {', '.join(df.columns.tolist()[:10])}... (and {len(df.columns)-10} more)")

In [ ]:
# Analyze per-token statistics FOR EACH LAYER
for layer_idx in range(num_layers):
    layer_name = f'layer_{layer_idx}'
    token_stats = token_stats_per_layer[layer_name]
    
    print(f"\n{'='*80}")
    print(f"LAYER {layer_idx} ANALYSIS")
    print(f"{'='*80}\n")
    
    # Compute derived metrics for each token
    token_analysis = {}
    
    for token_id, stats in token_stats.items():
        if stats['total_occurrences'] > 0:
            # Average entropy
            avg_entropy = stats['total_entropy'] / stats['total_occurrences']
            
            # Expert distribution (normalized)
            expert_distribution = stats['expert_counts'] / stats['expert_counts'].sum()
            
            # Most common expert
            most_common_expert = np.argmax(stats['expert_counts'])
            
            # Average expert size
            avg_expert_size = stats['expert_size_sum'] / stats['expert_counts'].sum()
            
            token_analysis[token_id] = {
                'avg_entropy': avg_entropy,
                'occurrences': stats['total_occurrences'],
                'expert_distribution': expert_distribution,
                'most_common_expert': most_common_expert,
                'avg_expert_size': avg_expert_size,
            }
    
    # Plot distribution of average expert sizes
    all_expert_sizes = np.array([a['avg_expert_size'] for a in token_analysis.values()])
    all_occurrences = np.array([a['occurrences'] for a in token_analysis.values()])
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    fig.suptitle(f'Layer {layer_idx} Token Routing Analysis', fontsize=16)
    
    # Unweighted histogram (by unique tokens)
    ax1 = axes[0]
    ax1.hist(all_expert_sizes, bins=50, alpha=0.7, edgecolor='black')
    ax1.axvline(x=config.expert_sizes[1][1], color='blue', linestyle='--', label=f'Small ({config.expert_sizes[1][1]})', linewidth=2)
    ax1.axvline(x=config.expert_sizes[0][1], color='red', linestyle='--', label=f'Large ({config.expert_sizes[0][1]})', linewidth=2)
    ax1.axvline(x=np.mean(all_expert_sizes), color='green', linestyle='--', label=f'Mean ({np.mean(all_expert_sizes):.0f})', linewidth=2)
    ax1.set_xlabel('Average Expert Size per Token')
    ax1.set_ylabel('Number of Unique Tokens')
    ax1.set_title('Distribution by Unique Tokens (Unweighted)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Weighted histogram (by token occurrences)
    ax2 = axes[1]
    ax2.hist(all_expert_sizes, bins=50, weights=all_occurrences, alpha=0.7, edgecolor='black', color='orange')
    ax2.axvline(x=config.expert_sizes[1][1], color='blue', linestyle='--', label=f'Small ({config.expert_sizes[1][1]})', linewidth=2)
    ax2.axvline(x=config.expert_sizes[0][1], color='red', linestyle='--', label=f'Large ({config.expert_sizes[0][1]})', linewidth=2)
    weighted_mean = np.average(all_expert_sizes, weights=all_occurrences)
    ax2.axvline(x=weighted_mean, color='green', linestyle='--', label=f'Weighted Mean ({weighted_mean:.0f})', linewidth=2)
    ax2.set_xlabel('Average Expert Size per Token')
    ax2.set_ylabel('Total Token Occurrences')
    ax2.set_title('Distribution by Token Occurrences (Weighted)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Pie chart - weighted breakdown
    ax3 = axes[2]
    large_expert_occurrences = sum(all_occurrences[all_expert_sizes > (config.n_embd * 4)/2])
    small_expert_occurrences = sum(all_occurrences[all_expert_sizes <= (config.n_embd * 4)/2])
    total_occurrences = large_expert_occurrences + small_expert_occurrences
    
    ax3.pie([large_expert_occurrences, small_expert_occurrences],
            labels=[f'Large experts\n(>{(config.n_embd * 4)/2})', f'Small experts\n(≤{(config.n_embd * 4)/2})'],
            autopct='%1.1f%%',
            colors=['red', 'blue'])
    ax3.set_title(f'Token Occurrences by Expert Size\n(Total: {total_occurrences:,} tokens)')
    
    plt.tight_layout()
    plt.show()
    
    # Statistics
    print(f"Average Expert Size Statistics (Unweighted):")
    print(f"  Mean: {np.mean(all_expert_sizes):.2f}")
    print(f"  Median: {np.median(all_expert_sizes):.2f}")
    print(f"  Std: {np.std(all_expert_sizes):.2f}")
    
    print(f"\nAverage Expert Size Statistics (Weighted by occurrences):")
    print(f"  Weighted Mean: {weighted_mean:.2f}")
    
    # Count how many tokens go to mostly large vs mostly small experts
    large_expert_tokens = sum(1 for s in all_expert_sizes if s > (config.n_embd * 4)/2)
    small_expert_tokens = sum(1 for s in all_expert_sizes if s <= (config.n_embd * 4)/2)
    print(f"\nUnique Token routing breakdown:")
    print(f"  Unique tokens routing mostly to LARGE experts: {large_expert_tokens} ({100*large_expert_tokens/len(all_expert_sizes):.1f}%)")
    print(f"  Unique tokens routing mostly to SMALL experts: {small_expert_tokens} ({100*small_expert_tokens/len(all_expert_sizes):.1f}%)")
    
    print(f"\nWeighted by occurrences:")
    print(f"  Token occurrences routed to LARGE experts: {large_expert_occurrences:,} ({100*large_expert_occurrences/total_occurrences:.1f}%)")
    print(f"  Token occurrences routed to SMALL experts: {small_expert_occurrences:,} ({100*small_expert_occurrences/total_occurrences:.1f}%)")

print("\n" + "="*80)
print("SUMMARY ACROSS ALL LAYERS")
print("="*80)

In [ ]:
import pandas as pd

# Extract the ratio from EXPERT_CONFIG (e.g., "5to1" -> 5, "23to1" -> 23)
ratio = int(EXPERT_CONFIG.split('to')[0])

dfs = {}
for seed in [42, 1223, 1337]:
    '''UNCOMMENT ME TO USE, MAKE SURE TO MATCH'''
    # checkpoint_dir = f"out-openwebtext/moe-8x2-variable-4x2944-4x128-seed{seed}
    # checkpoint_dir = f"gpt2_experiments/multiseed_23to1_20251102_203832/ratio23_lbl0.01_compute0.004_seed{seed}"
    # sweep_value = f"ratio23_lbl0.01_compute0.004_seed{seed}"
    # sweep_value = f"ratio5_lbl0.01_compute0.004_seed{seed}"
    dfs[seed] = pd.read_csv(f'analysis_csvs/{sweep_value}.csv')

In [7]:
import numpy as np
import pandas as pd

# Merge all three seeds
df = dfs[42].merge(dfs[1223], on='token', suffixes=('_42', '_1223'))
df = df.merge(dfs[1337], on='token')
df = df.rename(columns={'avg_size': 'avg_size_1337'})

# Calculate range (max - min) across all three seeds
df['max_size'] = df[['avg_size_42', 'avg_size_1223', 'avg_size_1337']].max(axis=1)
df['min_size'] = df[['avg_size_42', 'avg_size_1223', 'avg_size_1337']].min(axis=1)
df['range'] = df['max_size'] - df['min_size']
df['mean_size'] = df[['avg_size_42', 'avg_size_1223', 'avg_size_1337']].mean(axis=1)

# Use ratio to calculate bucket thresholds (scale from baseline of ratio=5)
# For ratio=5: tight=235, medium=470, wide=705
# Scale proportionally for other ratios
tight_threshold = 235
medium_threshold = 470
wide_threshold = 705

# Create different bucket sizes
df['all_exact'] = (
    np.isclose(df['avg_size_42'], df['avg_size_1223']) & 
    np.isclose(df['avg_size_1223'], df['avg_size_1337'])
)
exact_bucket = df[df['all_exact']].copy()
tight_bucket = df[df['range'] <= tight_threshold].copy()
medium_bucket = df[df['range'] <= medium_threshold].copy()
wide_bucket = df[df['range'] <= wide_threshold].copy()

print(f"Using ratio: {ratio}")
print(f"Bucket thresholds: tight={tight_threshold:.0f}, medium={medium_threshold:.0f}, wide={wide_threshold:.0f}\n")

print(f"Exact bucket:                {len(exact_bucket):,} tokens ({100*len(exact_bucket)/len(df):.1f}%)")
print(f"Tight bucket (±{tight_threshold:.0f}):  {len(tight_bucket):,} tokens ({100*len(tight_bucket)/len(df):.1f}%)")
print(f"Medium bucket (±{medium_threshold:.0f}): {len(medium_bucket):,} tokens ({100*len(medium_bucket)/len(df):.1f}%)")
print(f"Wide bucket (±{wide_threshold:.0f}):   {len(wide_bucket):,} tokens ({100*len(wide_bucket)/len(df):.1f}%)")

# Save to CSV with ratio-based filenames
exact_bucket.to_csv(f'analysis_csvs/owt_same_avg_size_tokens_ratio{ratio}_bucket_exact.csv', index=False)
tight_bucket.to_csv(f'analysis_csvs/owt_same_avg_size_tokens_ratio{ratio}_bucket_{tight_threshold:.0f}.csv', index=False)
medium_bucket.to_csv(f'analysis_csvs/owt_same_avg_size_tokens_ratio{ratio}_bucket_{medium_threshold:.0f}.csv', index=False)
wide_bucket.to_csv(f'analysis_csvs/owt_same_avg_size_tokens_ratio{ratio}_bucket_{wide_threshold:.0f}.csv', index=False)

print("\nSaved buckets to:")
print(f"  - analysis_csvs/owt_same_avg_size_tokens_ratio{ratio}_bucket_exact.csv")
print(f"  - analysis_csvs/owt_same_avg_size_tokens_ratio{ratio}_bucket_{tight_threshold:.0f}.csv")
print(f"  - analysis_csvs/owt_same_avg_size_tokens_ratio{ratio}_bucket_{medium_threshold:.0f}.csv")
print(f"  - analysis_csvs/owt_same_avg_size_tokens_ratio{ratio}_bucket_{wide_threshold:.0f}.csv")

Using ratio: 5
Bucket thresholds: tight=235, medium=470, wide=705

Exact bucket:                2,136 tokens (4.5%)
Tight bucket (±235):  13,079 tokens (27.4%)
Medium bucket (±470): 27,740 tokens (58.0%)
Wide bucket (±705):   45,064 tokens (94.3%)

Saved buckets to:
  - analysis_csvs/owt_same_avg_size_tokens_ratio5_bucket_exact.csv
  - analysis_csvs/owt_same_avg_size_tokens_ratio5_bucket_235.csv
  - analysis_csvs/owt_same_avg_size_tokens_ratio5_bucket_470.csv
  - analysis_csvs/owt_same_avg_size_tokens_ratio5_bucket_705.csv


In [ ]:
df_vanilla = pd.read_csv('analysis_csvs/moe-8x2-8x1536_expert_combinations.csv')

In [ ]:
exact_bucket_deduped = exact_bucket.drop_duplicates(subset='token')
df_vanilla_deduped = df_vanilla.drop_duplicates(subset='token')

result = exact_bucket_deduped.merge(df_vanilla_deduped, on='token', suffixes=('_exact', '_vanilla'))
result.to_csv('analysis_csvs/owt_same_avg_size_tokens_bucket_exact_vs_vanilla.csv', index=False)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('analysis_csvs/variable_experts_gpt2_data.csv')





In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('analysis_csvs/variable_experts_gpt2_data.csv')

df.plot(x='avg expert size', y='val/ce_loss', kind='scatter')
plt.show()

In [ ]:
df

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_csv('analysis_csvs/variable_experts_gpt2_data.csv')

# Create ratio labels manually based on the expert sizes
ratio_map = {
    'Baseline': '1:1',
    '2560x512': '5:1',
    '2688x384': '7:1',
    '2816x256': '11:1',
    '2944x128': '23:1'
}
df['ratio_label'] = df['Name'].map(ratio_map)

# Create numeric ratio for sorting/coloring
ratio_numeric = {
    'Baseline': 1,
    '2560x512': 5,
    '2688x384': 7,
    '2816x256': 11,
    '2944x128': 23
}
df['ratio'] = df['Name'].map(ratio_numeric)

fig, ax = plt.subplots(figsize=(10, 6))


ax.scatter(df['avg expert size'], df['val/ce_loss'], 
           c=df['ratio'], s=150, alpha=0.8, edgecolors='black', linewidth=1.5)


for idx, row in df.iterrows():
    ax.annotate(row['ratio_label'], 
                (row['avg expert size'], row['val/ce_loss']),
                xytext=(8, 8), textcoords='offset points',
                fontsize=11, fontweight='bold')


baseline_loss = df[df['Name'] == 'Baseline']['val/ce_loss'].values[0]
baseline_params = df[df['Name'] == 'Baseline']['avg expert size'].values[0]
ax.axhline(baseline_loss, color='gray', linestyle='--', alpha=0.5, linewidth=1)
ax.text(1500, baseline_loss + 0.002, 'Baseline', 
        ha='right', va='bottom', fontsize=9, color='gray')


ax.set_xlabel('Average Active Parameters per Token', fontsize=12, fontweight='bold')
ax.set_ylabel('Validation Loss', fontsize=12, fontweight='bold')
ax.set_title('MoE Efficiency: Loss vs Active Parameters', fontsize=14, fontweight='bold', pad=20)
ax.grid(True, alpha=0.3, linestyle=':', linewidth=0.5)
ax.set_ylim(3.22, 3.31)

plt.tight_layout()
plt.savefig('loss_vs_active_params.png', dpi=300, bbox_inches='tight')
plt.show()